In [ ]:
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import Adam

# MODEL DEFINITION

def discriminator_model(in_shape=(24,24,4)):
  model = Sequential()
    
  # convolution
  model.add(Conv2D(filters=128,
                   kernel_size=(2,2),
                   padding='same',
                   data_format='channels_last',
                   input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))

  # convolution again
  model.add(Conv2D(filters=128,
                   kernel_size=(2,2),
                   padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  # classifier
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  # compile model
  model.compile(loss='binary_crossentropy',
                optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
                metrics=['accuracy'])
  return model

def generator_model(latent_dim=100, pic_dim=6, num_channels=4):
  model = Sequential()
  num_copies = 128
  n_nodes = pic_dim**2 * num_copies

  # transform latent points to 6x6 image
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((pic_dim, pic_dim, num_copies)))

  # upsample to 12x12
  model.add(Conv2DTranspose(filters=num_copies,
                            kernel_size=(4,4),
                            strides=(2,2),
                            padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  # upsample to 24x24
  model.add(Conv2DTranspose(filters=num_copies,
                            kernel_size=(4,4),
                            strides=(2,2),
                            padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  # finish with tanh activation
  model.add(Conv2D(filters=num_channels,
                   kernel_size=(3,3),
                   activation='tanh',
                   padding='same'))

  return model

def define_gan(generator, discriminator):
  # lock weights for the discriminator
  discriminator.trainable = False

  # create joint network
  model = Sequential()
  model.add(generator)
  model.add(discriminator)
  model.compile(loss='binary_crossentropy',
                optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
  return model

In [ ]:
from PIL import Image
from tensorflow import convert_to_tensor
from tensorflow import expand_dims
import requests
import shutil
import os

# TRAINING DATA SETUP

def retrieve_data(num_points=1000):
    # setup
    folder_name = "images"
    image_url = lambda id : f"https://cryptopunks.app/public/images/cryptopunks/punk{id:04}.png"
    try:
        os.mkdir(folder_name)
    except: pass

    # request pngs
    for i in range(num_points):
        img_data = requests.get(image_url(i), stream=True)
        with open(f"{folder_name}/{i:04}.png", "wb") as f:
            shutil.copyfileobj(img_data.raw, f)

def load_real_data(folder_name, dataset_sz=10):
    all_images = []
    for i in range(dataset_sz):
        img_data = convert_to_tensor(Image.open(f"{folder_name}/{i:04}.png"), dtype='float32')
        img_data = (img_data - 127.5) / 127.5
        all_images.append(img_data)
    return convert_to_tensor(all_images)


In [ ]:
from numpy.random import randn

# NOISE GENERATION

def generate_random_latent_points(latent_dim, n):
    # generate a random array of size latent_dim (compressed 24x24)
    x_input = randn(latent_dim * n)
    x_input = x_input.reshape((n, latent_dim))
    return x_input

def generate_latent_points_from_image(id=0):
    # encode image to the latent space
    pass
    


In [ ]:
import random
from tensorflow import gather
from tensorflow import ones
from tensorflow import zeros

# TRAINING SAMPLES

def generate_real_samples(dataset, n):
    sample = random.sample(range(len(dataset)), n)
    new_x = gather(dataset, indices=sample)
    new_y = ones((n, 1))
    return new_x, new_y

def generate_fake_samples(generator, latent_dim, n):
    x_input = generate_random_latent_points(latent_dim, n)
    new_x = generator.predict(x_input)
    new_y = zeros((n, 1))
    return new_x, new_y

In [ ]:
# TRAIN MODELS

def train(g_model, d_model, gan_model, dataset, latent_dim, num_channels, epochs=100, n_batch=32):
    batch_per_epoch = dataset.shape[0]//n_batch
    half_batch = n_batch//2
    for i in range(epochs):
        for j in range(batch_per_epoch):
            x_real, y_real = generate_real_samples(dataset, half_batch)
            d_model.train_on_batch(x_real, y_real)
            x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_model.train_on_batch(x_fake, y_fake)
            x_gan = generate_random_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(x_gan, y_gan)
            
        if i%(epochs/10) == (epochs/10)-1:
            print(f"{int((i+1)/epochs * 100)}% complete")
    return g_model



In [ ]:
import numpy as np

# CREATE AND TRAIN MODELS

latent_dim = 100
num_channels = 4
discriminator = discriminator_model()
generator = generator_model(latent_dim, 6, num_channels)

# train the model
gan_model = define_gan(generator, discriminator)
dataset = load_real_data("images", 64)
model = train(generator, discriminator, gan_model, dataset, latent_dim, num_channels, epochs=100)


In [ ]:
import numpy as np
from tensorflow import expand_dims
# predict from input
latent_points = generate_random_latent_points(100, 1)
prediction = np.array(generator.predict(latent_points)).reshape(24,24,4)
image = Image.fromarray(prediction, mode="RGBA")
# print(prediction)
display(image.resize((400,400)))

# false_points = np.random.randn(24*24*4).reshape((1,24,24,4))
# true_points = np.array(Image.open(f"images/{2:04}.png")).reshape((1,24,24,4))
# prediction_f = discriminator.predict(false_points)
# prediction_t = discriminator.predict(true_points)
# print(prediction_f, prediction_t)